In [ ]:
import os
import sys
import pandas as pd
import numpy as np

In [ ]:
text_processing_scripts_path = os.path.abspath('TextProcessingScripts')
sys.path.append(text_processing_scripts_path)

scripts_path = os.path.abspath('Scripts')
sys.path.append(scripts_path)

In [ ]:
data_path = os.path.abspath('data')

In [ ]:
train_data = pd.read_csv(os.path.join(data_path, 'train.csv')).drop_duplicates()
train_data.index = range(len(train_data))

In [ ]:
aliased_snippet = []
for i in range(len(train_data)):
    aliased_snippet.append(train_data['snippet'][i].replace(train_data['company1'][i],'company1').replace(train_data['company2'][i],'company2'))
train_data['aliased_snippet'] = aliased_snippet

train_data['aliased_snippet'] = train_data['aliased_snippet'].str.lower()

In [ ]:
train_data.head()

In [ ]:
grouped_train_data = train_data.groupby(['company1','company2','is_parent'])['aliased_snippet'].apply(list)
grouped_train_data = grouped_train_data.to_frame().reset_index()

In [ ]:
grouped_train_data[grouped_train_data['company1']=='Google']['aliased_snippet'][414]